In [1]:
# 每次新建项目，先到运行：

# import zipfile
# import os
# def un_zip(file_name):
#     """unzip zip file"""
#     zip_file = zipfile.ZipFile(file_name)
#     for names in zip_file.namelist():
#         zip_file.extract(names,'work/')
#     zip_file.close()

# un_zip('/home/aistudio/data/data77240/pyeeg-master.zip')

# if os.path.isdir("work/data"):
#     pass
# else:
#     os.mkdir("work/data")

"""
每次运行环境，到终端执行如下两行命令（然后重启内核）：
cd ./work/pyeeg-master
python setup.py install
"""

'\n每次运行环境，到终端执行如下两行命令（然后重启内核）：\ncd ./work/pyeeg-master\npython setup.py install\n'

In [2]:
#加载飞桨、Numpy和相关类库
import numpy as np
import pyeeg as pe
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
import numpy as np
import pylab as pl
 
def wgn(x, snr):
    snr = 10**(snr/10.0)
    xpower = np.sum(x**2)/len(x)
    npower = xpower / snr
    return np.random.randn(len(x)) * np.sqrt(npower)
 
# t = np.arange(0, 1000) * 0.1
# x = np.sin(t)
# n = wgn(x, 5)
# xn = x+n # 增加了6dBz信噪比噪声的信号

In [4]:
# 切片
def piece(data):
    allTrail = []
    channelData = data.reshape((data.shape[0], 32, 7680))
    for trail in channelData:
        oneTrail = []
        for time in range(60):
            oneTrail.append(trail[:, time * 128 : (time + 1) * 128])
        allTrail.append(np.array(oneTrail))
    return np.array(allTrail).reshape((data.shape[0] * 60, 32, 128))

In [5]:
#特征提取
def feature_extraction(trainData):
    
    band = [4,8,13,16,30,45] #5 bands    band = [4,8,12,16,25,45]  
    sample_rate = 128 #Sampling rate of 128 Hz

    newData=[]
    for i in range(trainData.shape[0]):    #样本数
        channel_features=[]
        for j in range(32):
            X = trainData[i][j]
            Y = pe.bin_power(X, band, sample_rate)
            channel_features += list(Y[0])
        newData.append(np.array(channel_features))

    return np.array(newData)

In [6]:
def Gaussian_normalization(trainData, testData):
    """高斯归一化"""

    #训练样本归一化
    feature=trainData[:,:-1]
    mean=feature.mean(axis=0)
    std=feature.std(axis=0)
    feature-=mean
    feature/=std
    trainData[:,:-1] = feature

    #测试样本归一化
    feature=testData[:,:-1]
    feature-=mean
    feature/=std
    testData[:,:-1] = feature

    return trainData, testData

In [7]:
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

def trainModel(trainData, testData):
    
    trainLabel = trainData[:, -1]
    trainData = trainData[:, :-1]
    
    testLabel = testData[:, -1]
    testData = testData[:, :-1]
    
#     svmModel = svm.SVC()
#     svmModel.fit(data_training, valtrain) 
#     ypredict=svmModel.predict(data_testing)
#     print("SVMacc：",accuracy_score(ypredict,valtest)) 

    rfModel = RandomForestClassifier(n_estimators=512, n_jobs=16)
    rfModel.fit(trainData, trainLabel)
    ypredict = rfModel.predict(testData)
    acc = accuracy_score(ypredict,testLabel)
    print("acc = {:.2f}".format(acc))
    return acc

## 训练集、测试集的标签比例与总体样本的类标签比例相同
StratifiedShuffleSplit

In [ ]:
emotionDim = 0    #0 Valence 1 Arousal

multiples = 249    #数据增强multiple倍
snr = 5    #信噪比
ratio = 0.2    #测试集占的比例
num_splits = 10    #测试10次
randomState = 2021    #随机状态
allAcc = []    #记录num_splits次的准确率

dir = '/home/aistudio/data/data57707/1280x258050(32x8064+2).npy'

AllSub = np.load(dir)
feature = AllSub[: , :-2].reshape((1280, 32, 8064))

feature = feature[:, :, 384:]
feature = feature.reshape((1280, 245760))

label = np.where(AllSub[: , -2:] >= 5 , 1 , -1)
label = label[:, emotionDim]
del AllSub

sss = StratifiedShuffleSplit(n_splits=num_splits, test_size=ratio, random_state = randomState)#分成num_splits组，测试比例为ratio
num_split = 1    #第一次split
for train_index, test_index in sss.split(feature, label):
    X_train, X_test = feature[train_index], feature[test_index]#训练集对应的值
    y_train, y_test = label[train_index], label[test_index]#类别集对应的值
    
    
    unitSignal = piece(X_train)
    unitSignal = feature_extraction(unitSignal)
    unitLabel = []
    for idx in range(unitSignal.shape[0]):
        unitLabel.append(y_train[idx // 60])
    np.save('./work/trainSetFreq/{}.npy'.format(0), np.append(unitSignal, np.array(unitLabel).reshape((-1, 1)), axis = 1))
    
    unitSignal = piece(X_test)
    unitSignal = feature_extraction(unitSignal)
    unitLabel = []
    for idx in range(unitSignal.shape[0]):
        unitLabel.append(y_test[idx // 60])
    np.save('./work/testSetFreq.npy', np.append(unitSignal, np.array(unitLabel).reshape((-1, 1)), axis = 1))
    
    # 添加高斯白噪声，训练数据增强multiple倍
    for multiple in range(multiples):
        newSignal = []
        for signal in X_train:
            newChannelFeature = []
            channelFeature = signal.reshape((32, 7680))
            for channel in channelFeature:
                noise = wgn(channel, snr)
                noiseSignal = channel + noise
                newChannelFeature.append(noiseSignal)
            newChannelFeature = np.array(newChannelFeature).reshape(245760)
            newSignal.append(newChannelFeature)
        newSignal = np.array(newSignal)
        
        
        
        unitSignal = piece(newSignal)
        unitSignal = feature_extraction(unitSignal)
        unitLabel = []
        for idx in range(unitSignal.shape[0]):
            unitLabel.append(y_train[idx // 60])
        np.save('./work/trainSetFreq/{}.npy'.format(multiple + 1), np.append(unitSignal, np.array(unitLabel).reshape((-1, 1)), axis = 1))
        
    del X_test
    del X_train
    del y_train
    del y_test
    del unitSignal
    del unitLabel
    
    
    #加载训练集、测试集，开始训练、测试
    trainSet = []
    for multiple in range(multiples + 1):
        trainFreqFeature = np.load('./work/trainSetFreq/{}.npy'.format(multiple))
        trainSet.append(trainFreqFeature)
    trainSet = np.array(trainSet).reshape((-1, 161))
    testSet = np.load('./work/testSetFreq.npy')
    
    if num_split == 1:
        print('trainShape = ', trainSet.shape, 'testShape = ', testSet.shape)
    
    #------------------------------------是否归一化------------------------------------
    normal = True
    if normal:
        trainSet, testSet = Gaussian_normalization(trainSet, testSet)

    # 训练
    print('split{} ：'.format(num_split), end = '')
    num_split += 1
    acc = trainModel(trainSet, testSet)
    allAcc.append(acc)

In [ ]:
allAcc = np.array(allAcc)

maxIndex = allAcc.argmax()
print('meanAcc = ', allAcc.mean(),'num_split = ', maxIndex, 'maxAcc = ',allAcc.max(), 'var = ',allAcc.var())

In [ ]:
plt.ylim([0,1])
plt.xlabel('num_split')
plt.ylabel('acc')
plt.plot(range(1,11), allAcc)

In [ ]:
# normal = False meanAcc = 0.581
# normal = True meanAcc = 0.5811